# Query Example

In [61]:
from influxdb_client_3 import InfluxDBClient3, flight_client_options
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "vscode"
import os, datetime
import pyarrow.parquet as pq

In [62]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [63]:
query = '''SELECT * FROM machine_data WHERE time >= now() - 30m'''
table = client.query(query=query, language="influxql")
display(table)


pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
host: string
load: double
machineID: string
power: double
provider: string
temperature: double
topic: string
vibration: double
----
iox::measurement: [["machine_data","machine_data","machine_data","machine_data","machine_data",...,"machine_data","machine_data","machine_data","machine_data","machine_data"]]
time: [[2023-07-19 09:05:20.650630945,2023-07-19 09:05:20.650668170,2023-07-19 09:05:21.181076960,2023-07-19 09:05:21.651053150,2023-07-19 09:05:21.651134941,...,2023-07-19 09:35:10.204303387,2023-07-19 09:35:10.481225677,2023-07-19 09:35:11.205213437,2023-07-19 09:35:11.205328077,2023-07-19 09:35:11.481773219]]
host: [["de37f043d835","de37f043d835","de37f043d835","de37f043d835","de37f043d835",...,"de37f043d835","de37f043d835","de37f043d835","de37f043d835","de37f043d835"]]
load: [[85,68,89,68,85,...,200,99,200,75,99]]
machineID: [["machine3","machine2","machine1","machine2","machine3",...,"machine2","machine

In [52]:
pd = table.to_pandas().set_index('time')
fig = px.line(pd, y=["vibration"], title='raw vibration', color='machineID',  range_y=[0, 600] )
fig.show()

In [53]:
dict = table.to_pydict()
display(dict)

{'iox::measurement': ['machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machine_data',
  'machi

In [64]:
# PyArrow Aggregation
aggregation = table.group_by("machineID").aggregate([("vibration", "mean"), 
                                                     ("vibration", "max"), 
                                                     ("vibration", "min") ]).to_pandas()

display(aggregation)

,vibration_mean,vibration_max,vibration_min,machineID
0,85.042458,90.0,80.0,machine3
1,475.187709,600.0,80.0,machine2
2,85.061453,90.0,80.0,machine1


In [55]:
aggregation2 = table.group_by(["machineID", "time"]).aggregate([("vibration", "mean"), 
                                                     ("vibration", "max"), 
                                                     ("vibration", "min") ]).to_pandas()

display(aggregation2)

,vibration_mean,vibration_max,vibration_min,machineID,time
0,81.0,81.0,81.0,machine1,2023-07-19 08:53:02.622756506
1,90.0,90.0,90.0,machine3,2023-07-19 08:53:03.026186986
2,88.0,88.0,88.0,machine2,2023-07-19 08:53:03.026867134
3,82.0,82.0,82.0,machine1,2023-07-19 08:53:03.623069513
4,85.0,85.0,85.0,machine3,2023-07-19 08:53:04.026088061
...,...,...,...,...,...
5350,553.0,553.0,553.0,machine2,2023-07-19 09:22:47.554871272
5351,89.0,89.0,89.0,machine1,2023-07-19 09:22:47.938252192
5352,518.0,518.0,518.0,machine2,2023-07-19 09:22:48.555565112
5353,85.0,85.0,85.0,machine3,2023-07-19 09:22:48.555650278


In [65]:
pq.write_table(table, 'machine_data.parquet', version='2.6')

In [60]:
client.write_file(file="machine_data.parquet",database="python" , tag_columns=['machineID'], timestamp_column='time')